# PaddleOCR 다중 이미지(여러 장) 파이프라인 테스트

- 입력 소스/디렉터리 규칙은 `notebooks/ocr/paddleocr_test.ipynb`와 동일하게 유지합니다.
- `img_paths = [...]` 리스트로 여러 장을 처리합니다.
- 각 이미지마다 헤더 + 최종 JSON을 출력하고, 마지막에 `LabReportExtractor.merge_extractions`로 스마트 병합 수행.

> 형님 메모: PyCharm에서 노트북 UI로 열려야 정상입니다.
>
> todo pdf 도 전처리. 현재는 이미지 파일만 처리됨.


In [1]:
# 노트북 경로 부트스트랩 (원본 노트북과 동일 패턴)
import sys
from pathlib import Path

cwd = Path.cwd()
project_root = None
p = cwd

for _ in range(6):
    if (p / 'src').exists() and (p / 'pyproject.toml').exists():
        project_root = p
        break
    p = p.parent

if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f'✅ 프로젝트 루트: {project_root}')
print(f'✅ sys.path[0]: {sys.path[0]}')


✅ 프로젝트 루트: /home/aidan/projects/meow-chat
✅ sys.path[0]: /home/aidan/.pyenv/versions/3.12.12/lib/python312.zip


In [2]:
# 모듈 임포트 (원본 노트북과 동일 패턴)
import os

# 환경변수 설정 (반드시 settings import 전에 실행!)
os.environ['OCR_PROVIDER'] = 'paddle'
os.environ['OCR_USE_GPU'] = 'false'

from src.services.ocr.paddle_ocr import MyPaddleOCR
from src.services.preprocessing.image_preprocessor import ImagePreprocessor
from src.services.lab_extraction.line_preprocessor import LinePreprocessor
from src.services.lab_extraction.lab_table_extractor import LabTableExtractor
from src.services.lab_extraction.lab_report_extractor import LabReportExtractor
from src.settings import settings

print('✅ 모듈 임포트 완료')
print(f'   OCR Provider: {settings.ocr_provider}')
print(f'   OCR Use GPU: {settings.ocr_use_gpu}')


✅ 모듈 임포트 완료
   OCR Provider: paddle
   OCR Use GPU: False


In [3]:
# 인스턴스 생성
ocr_service = MyPaddleOCR(use_gpu=False, lang='korean')
image_preprocessor = ImagePreprocessor()
line_preprocessor = LinePreprocessor()
lab_table_extractor = LabTableExtractor()
# LabReportExtractor는 의존성을 직접 주입
lab_report_extractor = LabReportExtractor(
    line_preproc=line_preprocessor,
    extractor=lab_table_extractor
)

print('ocr_service=', type(ocr_service).__name__)
print('image_preprocessor=', type(image_preprocessor).__name__)
print('line_preprocessor=', type(line_preprocessor).__name__)
print('lab_table_extractor=', type(lab_table_extractor).__name__)
print('lab_report_extractor=', type(lab_report_extractor).__name__)


ocr_service= MyPaddleOCR
image_preprocessor= ImagePreprocessor
line_preprocessor= LinePreprocessor
lab_table_extractor= LabTableExtractor
lab_report_extractor= LabReportExtractor


In [4]:
# LLM 폴백 상태 확인 (.env에서 LAB_EXTRACTION_USE_LLM_METADATA 설정)
from src.settings import settings

if settings.lab_extraction_use_llm_metadata:
    if settings.openai_api_key:
        print(f'✅ LLM 메타데이터 폴백 활성화')
        print(f'   모델: {settings.openai_model_metadata}')
        print(f'   patient_name 추출 실패 시 LLM 사용')
    else:
        print('⚠️  LAB_EXTRACTION_USE_LLM_METADATA=true이지만 OPENAI_API_KEY가 없습니다.')
        print('   .env 파일에 OPENAI_API_KEY를 추가하세요.')
else:
    print('ℹ️  LLM 폴백 비활성화 (규칙 기반만 사용)')
    print('   활성화하려면 .env에 LAB_EXTRACTION_USE_LLM_METADATA=true 설정')



✅ LLM 메타데이터 폴백 활성화
   모델: gpt-4o-mini
   patient_name 추출 실패 시 LLM 사용


In [5]:
# 입력: 이미지 파일 / 폴더 / PDF 모두 지원
# - 파일이면: 이미지 1장 또는 PDF(페이지 N장)
# - 폴더면: 내부 이미지 파일들을 정렬해서 N장 처리
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Iterable

import io
from PIL import Image as PILImage

from src.utils.pdf import is_pdf, pdf_to_images

base_dir = project_root / 'notebooks' / 'ocr' / 'assets' / 'images'

SUPPORTED_IMAGE_EXTS = {'.jpg', '.jpeg', '.png', '.webp'}


@dataclass(frozen=True)
class WorkItem:
    source_name: str
    page_num: int | None  # PDF 페이지면 1부터, 이미지면 None
    pil_img: PILImage.Image


def build_work_items(file_paths: list[Path]) -> list[WorkItem]:
    """
    파일 Path 배열을 받아서 WorkItem 리스트로 변환.
    - 이미지 파일: 1개의 WorkItem (전처리 적용)
    - PDF 파일: N개의 WorkItem (페이지별로 분할, 전처리는 PDF → PIL 변환 시 적용)
    """
    items: list[WorkItem] = []

    for file_path in file_paths:
        if not file_path.exists():
            raise FileNotFoundError(f'파일이 존재하지 않습니다: {file_path}')

        if not file_path.is_file():
            raise RuntimeError(f'디렉터리는 지원하지 않습니다: {file_path}')

        # PDF 처리
        if is_pdf(file_path):
            pages = pdf_to_images(file_path, dpi=300)
            if not pages:
                raise RuntimeError(f'PDF에서 페이지를 추출하지 못했습니다: {file_path}')
            for i, pil_img in enumerate(pages):
                items.append(WorkItem(source_name=file_path.name, page_num=i + 1, pil_img=pil_img))

        # 이미지 처리 (전처리 적용)
        elif file_path.suffix.lower() in SUPPORTED_IMAGE_EXTS:
            raw_bytes = file_path.read_bytes()
            pre_bytes = image_preprocessor.process_bytes(raw_bytes, debug=False)
            pil_img = PILImage.open(io.BytesIO(pre_bytes))
            items.append(WorkItem(source_name=file_path.name, page_num=None, pil_img=pil_img))

        else:
            raise RuntimeError(f'지원하지 않는 파일 형식입니다: {file_path} (허용: PDF, 이미지)')

    return items

# 입력 파일 경로 리스트
file_paths = [
    base_dir / '20241221_0.jpg',
    base_dir / '20241221_1.jpg',
    base_dir / '20250119.PDF',
]

# 파일 리스트를 WorkItem 리스트로 변환
work_items = build_work_items(file_paths)

print(f'📄 file_paths={len(file_paths)} files')
print(f'📄 work_items={len(work_items)} items')
for wi in work_items[:10]:
    page_label = f'p={wi.page_num}' if wi.page_num else 'img'
    print(f' - {wi.source_name} ({page_label})')


📄 file_paths=3 files
📄 work_items=4 items
 - 20241221_0.jpg (img)
 - 20241221_1.jpg (img)
 - 20250119.PDF (p=1)
 - 20250119.PDF (p=2)


In [6]:
# 유틸: 문서 결과 생성/검증
from typing import Any, Dict
import json
import time

DocumentResult = Dict[str, Any]


def init_doc_result() -> DocumentResult:
    return {
        'hospital_name': None,
        'client_name': None,
        'patient_name': None,
        'inspection_date': None,
        'tests': [],
    }


def validate_doc_result(doc: DocumentResult) -> tuple[bool, str]:
    if not isinstance(doc, dict):
        return False, 'doc_result가 dict가 아닙니다.'
    tests = doc.get('tests', [])
    if not tests:
        return False, 'tests가 비어있습니다.'
    return True, f'tests={len(tests)}'


def print_header(idx: int, total: int, item: WorkItem, elapsed_s: float, raw_lines: int, tests: int):
    page_label = f'page={item.page_num}' if item.page_num else 'image'
    print('\n' + '=' * 80)
    print(f'🧾 [{idx + 1}/{total}] {item.source_name} ({page_label})')
    print(f'⏱️ elapsed: {elapsed_s:.2f}s | raw_lines: {raw_lines} | tests: {tests}')
    print('=' * 80)


In [7]:
# 다중 이미지/페이지 처리 루프
all_results: list[dict] = []

total = len(work_items)
for i, item in enumerate(work_items):
    t0 = time.time()

    pil_img = item.pil_img
    ocr_result = ocr_service.extract_text(pil_img)

    raw_lines: list[str] = []
    if ocr_result and getattr(ocr_result, 'data', None) and getattr(ocr_result.data, 'items', None):
        for ocr_item in ocr_result.data.items:
            raw_lines.extend(getattr(ocr_item, 'rec_texts', []) or [])
    raw_text = '\n'.join(raw_lines)

    doc_result = init_doc_result()
    try:
        if ocr_result and ocr_result.data and ocr_result.data.items:
            page = ocr_result.data.items[0]
            lined_data = line_preprocessor.extract_and_group_lines(page)
            extracted, _intermediates = lab_table_extractor.extract_from_lines(lined_data, return_intermediates=True)
            if extracted:
                doc_result.update(extracted)
    except Exception as e:
        doc_result['__error__'] = str(e)

    elapsed = time.time() - t0
    is_valid, reason = validate_doc_result(doc_result)

    print_header(
        i,
        total,
        item,
        elapsed_s=elapsed,
        raw_lines=len(raw_lines),
        tests=len(doc_result.get('tests', []) or []),
    )
    print(f'valid={is_valid} ({reason})')
    print(json.dumps(doc_result, ensure_ascii=False, indent=2))

    all_results.append(
        {
            'source_name': item.source_name,
            'page_num': item.page_num,
            'elapsed_s': elapsed,
            'raw_text': raw_text,
            'doc_result': doc_result,
            'valid': is_valid,
            'valid_reason': reason,
        }
    )


Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.
/home/aidan/projects/meow-chat/.venv/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete t

[DEBUG] tokens_with_geometry: count=155
[
  {
    "text": "SAN",
    "confidence": 0.9959301948547363,
    "y_top": 1,
    "y_bottom": 26,
    "y_center": 14,
    "raw_h": 25,
    "x_left": 2,
    "x_right": 48
  },
  {
    "text": "백산동물병원",
    "confidence": 0.9841563701629639,
    "y_top": 0,
    "y_bottom": 39,
    "y_center": 20,
    "raw_h": 39,
    "x_left": 95,
    "x_right": 329
  },
  {
    "text": "백산동물명원",
    "confidence": 0.8986464142799377,
    "y_top": 0,
    "y_bottom": 20,
    "y_center": 10,
    "raw_h": 20,
    "x_left": 341,
    "x_right": 505
  },
  {
    "text": "전료기록-아전경(양의)",
    "confidence": 0.7921392321586609,
    "y_top": 0,
    "y_bottom": 27,
    "y_center": 14,
    "raw_h": 27,
    "x_left": 1054,
    "x_right": 1353
  },
  {
    "text": "서울특별시 강남구 역삼동 725-15 Tel) 02-1644-5653",
    "confidence": 0.9349622130393982,
    "y_top": 7,
    "y_bottom": 49,
    "y_center": 28,
    "raw_h": 42,
    "x_left": 335,
    "x_right": 849
  },
  {
    "text": "Laborato

In [8]:
# 전체 요약
print('\n' + '#' * 80)
print('SUMMARY')
print('#' * 80)
for r in all_results:
    tests = len((r.get('doc_result') or {}).get('tests', []) or [])
    page = r.get('page_num')
    page_label = f' p={page}' if page else ''
    print(
        f"- {r['source_name']}{page_label}: valid={r['valid']} tests={tests} elapsed={r['elapsed_s']:.2f}s"
    )



################################################################################
SUMMARY
################################################################################
- 20241221_0.jpg: valid=True tests=23 elapsed=12.76s
- 20241221_1.jpg: valid=True tests=9 elapsed=6.47s
- 20250119.PDF p=1: valid=True tests=25 elapsed=20.50s
- 20250119.PDF p=2: valid=True tests=20 elapsed=17.44s


In [9]:
# (추가) 각 문서별 상세 출력 (헤더 + 표 형식 JSON)
print('\n' + '=' * 80)
print('📋 각 문서별 상세 결과')
print('=' * 80)

for idx, r in enumerate(all_results):
    doc = r.get('doc_result') or {}
    tests = doc.get('tests', []) or []

    page = r.get('page_num')
    page_label = f' p={page}' if page else ''

    print(f"\n🧾 Step {idx + 1}: {r['source_name']}{page_label}")
    print(f"⏱️ elapsed: {r['elapsed_s']:.2f}s | tests: {len(tests)}")
    print()

    # 메타 정보 출력
    print(f"🏥 hospital_name    : {doc.get('hospital_name') or '(None)'}")
    print(f"👤 client_name      : {doc.get('client_name') or '(None)'}")
    print(f"🐾 patient_name     : {doc.get('patient_name') or '(None)'}")
    print(f"🗓  inspection_date : {doc.get('inspection_date') or '(None)'}")
    print()

    if tests:
        # 표 헤더
        print('code        | value | unit    | reference_min | reference_max')
        print('------------+-------+---------+---------------+--------------')

        # 각 테스트 항목
        for t in tests:
            code = (t.get('code') or 'UNKNOWN')[:12].ljust(12)
            value = str(t.get('value') or '')[:5].rjust(5)
            unit = (t.get('unit') or 'UNKNOWN')[:7].ljust(7)
            ref_min = str(t.get('reference_min') or 'UNKNOWN')[:13].rjust(13)
            ref_max = str(t.get('reference_max') or 'UNKNOWN')[:13].rjust(13)

            print(f'{code}| {value} | {unit} | {ref_min} | {ref_max}')
    else:
        print('(추출된 테스트 항목 없음)')



📋 각 문서별 상세 결과

🧾 Step 1: 20241221_0.jpg
⏱️ elapsed: 12.76s | tests: 23

🏥 hospital_name    : (None)
👤 client_name      : Patient: 양희
🐾 patient_name     : 양희
🗓  inspection_date : 2024-12-21

code        | value | unit    | reference_min | reference_max
------------+-------+---------+---------------+--------------
LYMPHO      |  17.1 | %       |       UNKNOWN |       UNKNOWN
MONO        |   6.3 | %       |       UNKNOWN |       UNKNOWN
EOS         |   3.7 | %       |       UNKNOWN |       UNKNOWN
BASO        |   0.4 | %       |       UNKNOWN |       UNKNOWN
NEUT        | 10.09 | K/µL    |          1.15 |         10.29
LYMPH       |  2.38 | K/µL    |          0.92 |          6.88
MONO        |  0.88 | K/µL    |          0.05 |          0.67
EOS         |  0.51 | K/µL    |          0.17 |          1.57
BASO        |  0.06 | K/µL    |          0.01 |          0.26
PLT         | 296.0 | K/µL    |         151.0 |         600.0
MPV         |  22.2 | fL      |       UNKNOWN |       UNKNOWN
PCT

In [10]:
# (스마트 병합) LabReportExtractor.merge_extractions 사용
# - 동일 검사일/header_shape이면 병합
# - 별개 검사지면 분리 유지
# - 중복 code+unit 자동 제거

doc_results = [r['doc_result'] for r in all_results]
merge_envelope = lab_report_extractor.merge_extractions(doc_results)

print('\n' + '=' * 80)
print('🔀 SMART MERGE (LabReportExtractor.merge_extractions)')
print('=' * 80)
print(f'입력: {len(doc_results)}개 문서')
print(f'병합 후: {merge_envelope.meta.merged_len}개 문서')
print(f'빈 문서 제외: {merge_envelope.meta.pruned_empty}개')
print(f'중복 제거: {merge_envelope.meta.before_dedup} → {merge_envelope.meta.after_dedup} tests')
print()

for idx, merged_doc in enumerate(merge_envelope.data.merged):
    tests = merged_doc.get('tests', [])
    print(f'\n📄 병합 문서 #{idx + 1}')
    print(f'🏥 hospital_name    : {merged_doc.get("hospital_name") or "(None)"}')
    print(f'👤 client_name      : {merged_doc.get("client_name") or "(None)"}')
    print(f'🐾 patient_name     : {merged_doc.get("patient_name") or "(None)"}')
    print(f'🗓  inspection_date : {merged_doc.get("inspection_date") or "(None)"}')
    print(f'📊 tests count      : {len(tests)}')
    print()

    if tests:
        print('code        | value | unit    | reference_min | reference_max')
        print('------------+-------+---------+---------------+--------------')
        for t in tests:
            code = (t.get('code') or 'UNKNOWN')[:12].ljust(12)
            value = str(t.get('value') or '')[:5].rjust(5)
            unit = (t.get('unit') or 'UNKNOWN')[:7].ljust(7)
            ref_min = str(t.get('reference_min') or 'UNKNOWN')[:13].rjust(13)
            ref_max = str(t.get('reference_max') or 'UNKNOWN')[:13].rjust(13)
            print(f'{code}| {value} | {unit} | {ref_min} | {ref_max}')



🔀 SMART MERGE (LabReportExtractor.merge_extractions)
입력: 4개 문서
병합 후: 2개 문서
빈 문서 제외: 0개
중복 제거: 77 → 66 tests


📄 병합 문서 #1
🏥 hospital_name    : SAN BAEK 백산동물병원
👤 client_name      : Patient: 양희
🐾 patient_name     : 양희
🗓  inspection_date : 2024-12-21
📊 tests count      : 23

code        | value | unit    | reference_min | reference_max
------------+-------+---------+---------------+--------------
LYMPHO      |  17.1 | %       |       UNKNOWN |       UNKNOWN
MONO        |   6.3 | %       |       UNKNOWN |       UNKNOWN
EOS         |   3.7 | %       |       UNKNOWN |       UNKNOWN
BASO        |   0.4 | %       |       UNKNOWN |       UNKNOWN
NEUT        | 10.09 | K/µL    |          1.15 |         10.29
LYMPH       |  2.38 | K/µL    |          0.92 |          6.88
MONO        |  0.88 | K/µL    |          0.05 |          0.67
EOS         |  0.51 | K/µL    |          0.17 |          1.57
BASO        |  0.06 | K/µL    |          0.01 |          0.26
PLT         | 296.0 | K/µL    |         151.0